## Modelo de predicción de sentimientos en frases

In [25]:
import pandas as pd
import nltk
import string
import re

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import stopwords


from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer,LancasterStemmer
from transformers import AutoTokenizer
import emoji

In [6]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]    |   Pac

True

In [59]:
train = pd.read_csv('text_classification_train.csv',sep=',')
train.head(30)

,text,emotion,id
0,My favourite food is anything I didn't have to...,27,eebbqej
1,"Now if he does off himself, everyone will thin...",27,ed00q6i
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
3,To make her feel threatened,14,ed7ypvh
4,Dirty Southern Wankers,3,ed0bdzj
5,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,26,edvnz26
6,Yes I heard abt the f bombs! That has to be wh...,15,ee3b6wu
7,We need more boards and to create a bit more s...,"8,20",ef4qmod
8,Damn youtube and outrage drama is super lucrat...,0,ed8wbdn
9,It might be linked to the trust factor of your...,27,eczgv1o


Se observa que en el dataframe existen frases que contienen dos emociones por lo que para poder trabajar con esta informacion se crea una funcion que en primer lugar revisa la existencia de dos emociones en una fila y luego agrega dos filas con el mismo texto y id con las diferentes emociones.

In [15]:
def separar_emociones(data):
    for i in range(0,data.shape[0]):
        if ',' in data.emotion[i]:
            aux = data.emotion[i].split(',')
            aux_data = pd.DataFrame({'text':[data.text[i],data.text[i]],'emotion':[aux[0],aux[1]],'id':[data.id[i],data.id[i]]})
            data = pd.concat([data,aux_data],ignore_index=True)
            data = data.drop(data.index[i])
    return data

In [16]:
train = separar_emociones(train)

In [17]:
#https://stackoverflow.com/questions/72560920/how-to-remove-emojis-from-panda-dataframe-column-works-on-individual-string-but
def remove_emoji(string):
   emoji_pattern = re.compile("["
                       u"\U0001F600-\U0001F64F"  # emoticons
                       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                       u"\U0001F680-\U0001F6FF"  # transport & map symbols
                       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       "]+", flags=re.UNICODE)
   return emoji_pattern.sub(r'', string)

In [48]:
def emoji2concat_description(text):
    emoji_list = emoji.emoji_list(text)
    ret = emoji.replace_emoji(text, replace='').strip()
    for json in emoji_list:
        this_desc = ' '.join(emoji.EMOJI_DATA[json['emoji']]['en'].split('_')).strip(':')
        ret += ' ' + this_desc
    return ret

In [51]:
emoji2concat_description("hola 😭")

'hola loudly crying face'

In [56]:
def limpieza_text(texto):
    texto = emoji2concat_description(texto)
    # dejar todo el texto en minuscula
    token_list = word_tokenize(texto.lower())
    token_list_no_punctuation =  [token for token in token_list if token not in string.punctuation]
    #eliminar stop words, palabras que no aportan
    
    filtered_tokens = [token for token in token_list_no_punctuation if token not in stopwords.words('english')]
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    lemmatizer2 = LancasterStemmer()
    lemmatized_tokens2 = [lemmatizer2.stem(token) for token in lemmatized_tokens]    
        # Join the tokens back into a string
    processed_text = ' '.join(lemmatized_tokens2)
    return processed_text


In [60]:
train['text'] = train['text'].apply(limpieza_text)

In [61]:
train.head(30)

,text,emotion,id
0,favourit food anyth n't cook,27,eebbqej
1,everyon think he laugh screwing peopl instead ...,27,ed00q6i
2,fuck bayless iso,2,eezlygj
3,mak feel threatened,14,ed7ypvh
4,dirty southern wank,3,ed0bdzj
5,omg peyton n't good enough help u playoff dumb...,26,edvnz26
6,ye heard abt f bomb thank reply hubby anxy wai...,15,ee3b6wu
7,nee board cre bit spac nam ’ good,"8,20",ef4qmod
8,damn youtub out dram sup lucr reddit,0,ed8wbdn
9,might link trust fact friend,27,eczgv1o
